In [4]:
import sys
sys.path.append('Scripts')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from utils import path, batchVisualization, createInputFigure
from SSVEPDataset import SSVEPDataset
from classes import *
from SSVEPDataloader import *

from sklearn import preprocessing
import os
import time
import copy

In [5]:
%load_ext autoreload
%autoreload 2

In [3]:
def read_data(dataframe, path, ind):
    return torch.tensor(pd.read_csv(path/dataframe.loc[ind]['path'],  sep = ' ', usecols = range(16), names = range(1, 17), dtype = float).values)

#dataPath = path('/home/roboto/Documents/GitHub/ICDeepLearning/Data')
dataPath = path('C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data')
data = pd.read_csv('SSVEPDataset.csv')
dataset = SSVEPDataset(data, dataPath)

batchSize = 1
dataloaders, datasetsSizes = SSVEPDataloaders(dataset, batchSize)
# example = next(iter(dataloaders['train']))['series']

Directory C:\Users\Patrick\Documents\GitHub\ICDeepLearning\Data loaded successfully.


In [1]:
def convOutputSize(i, k, p, s):
    return int(np.floor((i + 2*p - k)/s + 1))

def deconvOutputsize(i, k, p, s):
    if s == 1 and p == 0:
        return i + k - 1
    if (not k%2 == 0) and s == 1 and p == np.floor(k/2):
        return i
    if (i - k)%2 == 0:
        return 

def maxpoolOutputShape(i, k, p, s, d):
    return np.floor((i + 2*p - d*(k - 1) - 1)/s + 1)

In [17]:
parameters = [16, 5, 2, 1]

print(convOutputSize(*parameters))
#print(deconvOutputsize(*parameters))
#print(maxpoolOutputShape(512, 3, 1, 2, 1))
#print(maxpoolOutputShape(16, 3, 1, 1, 1))

16


In [14]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            ConvBlock(1, 16, kernel_size = [3, 3], padding = [1, 1]),
            ConvBlock(16, 32, kernel_size = [3, 3], padding = [1, 1]),
            ConvBlock(32, 64, kernel_size = [3, 3], padding = [1, 1]),
            nn.MaxPool2d(kernel_size = [3, 3], stride = [2, 1], padding = [1, 1])
        )
        
        self.decoder = nn.Sequential(
            nn.UpsamplingNearest2d(scale_factor = (2, 1)),
            DeconvBlock(64, 32, kernel_size = [3, 3], padding = [1, 1]),
            DeconvBlock(32, 16, kernel_size = [3, 3], padding = [1, 1]),
            DeconvBlock(16, 1, kernel_size = [3, 3], padding = [1, 1])
        )
        
    def forward(self, X):
        X = self.encoder(X)
        X = self.decoder(X)
        return X

In [13]:
model = Autoencoder().double()
model(next(iter(dataloaders['train']))['series'])

torch.Size([1, 64, 256, 16])


tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1008],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0043],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0587,  ..., 0.0000, 0.0415, 0.6923],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.9468, 1.1862],
          [0.0000, 0.0000, 0.0000,  ..., 0.2293, 2.4679, 1.0508]]]],
       dtype=torch.float64, grad_fn=<ReluBackward1>)

In [15]:
# def train(model, dataloader, num_epochs = 5, batch_size = 16, learning_rate = 1e-3):
#     torch.manual_seed(42)
#     criterion = nn.MSELoss()
#     optimizer = torch.optim.Adam(model.parameters(),
#                                  lr = learning_rate,
#                                  weight_decay = 1e-5)
    
#     train_loader = dataloader['train']
    
#     outputs = []
#     for epoch in range(num_epochs):
#         for data in train_loader:
#             img = data['series']
#             recon = model(img)
#             loss = criterion(recon, img)
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()
            
#         print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
#         outputs.append((epoch, img, recon),)
    
#     return outputs

def train(model, dataloaders, criterion, optimizer, scheduler, device,
          num_epochs = 25, learningRate = 1e-3):
    since = time.time()
    bestModel = copy.deepcopy(model.state_dict())
    bestLoss = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and a validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            runningLoss = 0.0
            runningCorrects = 0
            
            for inputs in dataloaders[phase]:
                inputs = inputs['series'].to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, inputs)
                    
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                runningLoss += loss.item() * inputs.size(0)
            if phase == 'train':
                scheduler.step()
                
            epochLoss = runningLoss/datasetsSizes[phase]
            
            print('{} Loss: {:.4f}'.format(phase, epochLoss))
            
            if phase == 'val' and epochLoss > bestLoss:
                bestLoss = epochLoss
                best_model = copy.deepcopy(model.state_dict())
                
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(bestLoss))
    
    model.load_state_dict(best_model)
    return model

In [20]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

model = Autoencoder().double()
model = model.to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr = 0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

model = train(model, dataloaders, criterion, optimizer, scheduler, device,
              num_epochs = 10)

Epoch 0/9
----------
train Loss: 0.0294
val Loss: 0.0234

Epoch 1/9
----------
train Loss: 0.0233
val Loss: 0.0217

Epoch 2/9
----------
train Loss: 0.0222
val Loss: 0.0219

Epoch 3/9
----------
train Loss: 0.0216
val Loss: 0.0220

Epoch 4/9
----------
train Loss: 0.0212
val Loss: 0.0240

Epoch 5/9
----------
train Loss: 0.0206
val Loss: 0.0226

Epoch 6/9
----------
train Loss: 0.0269
val Loss: 0.0267

Epoch 7/9
----------
train Loss: 0.0232
val Loss: 0.0265

Epoch 8/9
----------
train Loss: 0.0228
val Loss: 0.0255

Epoch 9/9
----------
train Loss: 0.0225
val Loss: 0.0258

Training complete in 12m 36s
Best val Loss: 0.026675


In [ ]:
model = Autoencoder().double()
max_epochs = 20
outputs = train(model, dataloaders, num_epochs = max_epochs)
outputs